# **Fowl Escapades**
#### This notebook shows one of the possible solutions of the 'Fowl Escapades' challenge on Zindi.This solution took 16th place with a Log Loss of 2.18.
### **Data used:**
#### Test.zip, Train.zip and CSV files (Train.csv, SampleSubmission.csv) from Zindi. All data is available on Zindi: https://zindi.africa/competitions/fowl-escapades/data
#### External data was not used.
### **Output data:**
#### birds_276.csv - features of training dataset
#### birds_test_276.csv -  features of test dataset
#### model_ens_3_all.h5 - model trained on 100 epochs
#### submission_starter_notebook_8.csv - file following the sample submission format with model's predicted probabilities

### All data should be stored in the folder on path

In [4]:
path = "Birds/"

In [2]:
import os
import pandas as pd
import numpy as np
import librosa
import csv
import tensorflow

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

In [5]:
train = pd.read_csv(os.path.join(path, "Train.csv"))

In [7]:
# Generate labels for train
Y_train = np.array(train['common_name'].values)

In [8]:
# Preparing paths for training data
train['file_name'] = 'Train/'+train['ID']+'.mp3'

In [9]:
train.head()

,ID,common_name,file_name
0,MBMG2C,Ring-necked Dove,Train/MBMG2C.mp3
1,K8LJSB,Ring-necked Dove,Train/K8LJSB.mp3
2,OGD9L6,Ring-necked Dove,Train/OGD9L6.mp3
3,581PCQ,Ring-necked Dove,Train/581PCQ.mp3
4,P91M1F,Ring-necked Dove,Train/P91M1F.mp3


## **Feature extraction**
#### magnitude - magphase separate a complex-valued spectrogram D (stft) into its magnitude (S) and phase (P) components (65)
#### tonnetz - tonal centroid features (6)
#### mfcc - Mel-frequency cepstral coefficients (number of MFCCs: 26)
#### logmel - a mel-scaled spectrogram (number of mels: 26)
#### chroma - a chromagram from a waveform or power spectrogram (12)
#### cens - computes the chroma variant “Chroma Energy Normalized” (CENS) (3 - max, min and mean values)
#### Total 276 features (mean and std)

In [10]:
# Normalization
def minMaxNormalize(arr):
    mn = np.min(arr)
    mx = np.max(arr)
    return (arr-mn)/(mx-mn)

In [11]:
def features_extraction(paths):
  X_train = []
  for pth in paths:
    audio_data, sample_rate = librosa.load(os.path.join(path,pth))

    stft = librosa.stft(audio_data, n_fft=128, hop_length=1024)
    magnitude, phase = librosa.magphase(stft)

    tonnetz = librosa.feature.tonnetz(audio_data)
    
    mfcc = librosa.feature.mfcc(audio_data, n_mfcc = 26, hop_length = 1024, n_fft = 512)

    logmel = librosa.feature.melspectrogram(audio_data, hop_length = 1024, n_fft = 512, n_mels=26)

    chroma = librosa.feature.chroma_stft(audio_data, hop_length = 1024, n_fft = 512)

    cens = librosa.feature.chroma_cens(audio_data, hop_length = 1024)
    cens_arr = []
    cens_mean = cens.mean(axis = 0)
    cens_arr.append(max(cens_mean))
    cens_arr.append(min(cens_mean))
    cens_arr.append(cens_mean.mean(axis=0))
    cens_std = cens.std(axis = 0)
    cens_arr.append(max(cens_std))
    cens_arr.append(min(cens_std))
    cens_arr.append(cens_std.mean(axis=0))
    cens_arr = np.array(cens_arr)

    features_all = np.concatenate((magnitude.mean(axis = 1), magnitude.std(axis = 1), tonnetz.mean(axis = 1), tonnetz.std(axis = 1),
                                   mfcc.mean(axis = 1), mfcc.std(axis = 1), minMaxNormalize(logmel.mean(axis = 1)),
                                   minMaxNormalize(logmel.std(axis = 1)), chroma.mean(axis = 1), chroma.std(axis = 1), cens_arr), axis = 0)
    
    X_train.append(features_all)
  return X_train

In [0]:
X_train = features_extraction(train['file_name'].values)

In [0]:
#X_train = np.array(features_extraction(train['file_name'].values[:50]))

In [0]:
# Write train features to csv
with open(os.path.join(path, "birds_276.csv"), "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(X_train)

In [12]:
# Read train features from csv
# X_train = np.array(pd.read_csv(os.path.join(path, "birds_276.csv"), header=None))

In [13]:
X_train.shape

(1857, 276)

In [14]:
# Mix training data
n_samples = len(Y_train)
order = np.array(range(n_samples))
np.random.shuffle(order)
X_train_unorder = X_train[order]
Y_train_unorder = Y_train[order]

In [15]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [16]:
# Transform labels to arrays of size 40: [0, 0, 1, ..., 0, 0]
lb = LabelEncoder()
Y_train_unorder = np_utils.to_categorical(lb.fit_transform(Y_train_unorder))
# Split data on training and validation data
# X_train_new, X_validation, y_train_new, y_validation = train_test_split(X_train_unorder, Y_train_unorder, test_size = 0.2, random_state = 42)

In [21]:
# Reshape training data
X_train_unorder = np.array(X_train_unorder).reshape(X_train_unorder.shape[0], X_train_unorder.shape[1], 1)
# X_train_new = np.array(X_train_new).reshape(X_train_new.shape[0], X_train_new.shape[1], 1)
# X_validation = np.array(X_validation).reshape(X_validation.shape[0], X_validation.shape[1], 1)

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.layers import MaxPooling1D, Conv1D, GlobalAveragePooling1D, Flatten, Concatenate
from keras.optimizers import Adam, SGD
from sklearn import metrics
import random
from keras import optimizers

from keras.models import load_model, Model, save_model
from keras.layers import LeakyReLU

import keras.layers as L
from sklearn.metrics import log_loss
from keras import regularizers

In [19]:
# Last model on submit
inputs1 = L.Input((128,1))
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs1)
conv1 = Dropout(0.25)(conv1)
conv1 = MaxPooling1D(pool_size=2)(conv1)
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(conv1)
conv1 = Dropout(0.25)(conv1)
conv1 = MaxPooling1D(pool_size=2)(conv1)
conv1 = Conv1D(filters=16, kernel_size=3, activation='relu')(conv1)
conv1 = Dropout(0.25)(conv1)
conv1 = MaxPooling1D(pool_size=2)(conv1)
conv1 = Conv1D(filters=8, kernel_size=6, activation='relu')(conv1)
conv1 = Dropout(0.25)(conv1)
#conv1 = MaxPooling1D(pool_size=2)(conv1)
conv1 = Flatten()(conv1)
	# head 2
inputs2 = L.Input((6,1))
conv2 = Conv1D(filters=2, kernel_size=3, activation='relu')(inputs2)
conv2 = Dropout(0.15)(conv2)
#conv2 = MaxPooling1D(pool_size=2)(conv2)
conv2 = Flatten()(conv2)
	# head 3
inputs3 = L.Input((142,1))
conv3 = Conv1D(filters=128, kernel_size=3, activation='relu')(inputs3)
conv3 = Dropout(0.15)(conv3)
conv3 = MaxPooling1D(pool_size=3)(conv3)
conv3 = Conv1D(filters=64, kernel_size=3, activation='relu')(conv3)
conv3 = Dropout(0.15)(conv3)
conv3 = MaxPooling1D(pool_size=2)(conv3)
conv3 = Conv1D(filters=16, kernel_size=3, activation='relu')(conv3)
conv3 = Dropout(0.15)(conv3)
conv3 = MaxPooling1D(pool_size=2)(conv3)
conv3 = Conv1D(filters=8, kernel_size=6, activation='relu')(conv3)
conv3 = Dropout(0.15)(conv3)
#conv3 = MaxPooling1D(pool_size=2)(conv3)
conv3 = Flatten()(conv3)
	# merge
merged = Concatenate(axis=1)([conv1, conv2, conv3])
	# interpretation
dense1 = Dense(120, activation='relu')(merged)
outputs = Dense(40, activation='softmax')(dense1)
model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)

model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 1)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 142, 1)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 126, 64)      256         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)    

In [0]:
# Train with validation data
#model.fit([X_train_new[:,:128], X_train_new[:,128:134], X_train_new[:,134:276]], y_train_new, batch_size=64, epochs=100,validation_data=([X_validation[:,:128], X_validation[:,128:134], X_validation[:,134:276]],y_validation))

Train on 1485 samples, validate on 372 samples
Epoch 1/100
1485/1485 [==============================] - 2s 2ms/step - loss: 3.7577 - accuracy: 0.0404 - val_loss: 3.6537 - val_accuracy: 0.0376
Epoch 2/100
1485/1485 [==============================] - 2s 1ms/step - loss: 3.5942 - accuracy: 0.0747 - val_loss: 3.5614 - val_accuracy: 0.0699
Epoch 3/100
1485/1485 [==============================] - 2s 1ms/step - loss: 3.4490 - accuracy: 0.1037 - val_loss: 3.4007 - val_accuracy: 0.1344
Epoch 4/100
1485/1485 [==============================] - 2s 1ms/step - loss: 3.2877 - accuracy: 0.1360 - val_loss: 3.2096 - val_accuracy: 0.1613
Epoch 5/100
1485/1485 [==============================] - 2s 1ms/step - loss: 3.1158 - accuracy: 0.1522 - val_loss: 3.0910 - val_accuracy: 0.1855
Epoch 6/100
1485/1485 [==============================] - 2s 1ms/step - loss: 2.9753 - accuracy: 0.1737 - val_loss: 2.9653 - val_accuracy: 0.1962
Epoch 7/100
1485/1485 [==============================] - 2s 1ms/step - loss: 2.8960

In [0]:
# Training
model.fit([X_train_unorder[:,:128], X_train_unorder[:,128:134], X_train_unorder[:,134:276]], Y_train_unorder, batch_size=64, epochs=100)

Epoch 1/140
1857/1857 [==============================] - 3s 2ms/step - loss: 3.8680 - accuracy: 0.0350
Epoch 2/140
1857/1857 [==============================] - 3s 1ms/step - loss: 3.5743 - accuracy: 0.0749
Epoch 3/140
1857/1857 [==============================] - 3s 1ms/step - loss: 3.4100 - accuracy: 0.1325
Epoch 4/140
1857/1857 [==============================] - 3s 1ms/step - loss: 3.2366 - accuracy: 0.1481
Epoch 5/140
1857/1857 [==============================] - 3s 1ms/step - loss: 3.0814 - accuracy: 0.1605
Epoch 6/140
1857/1857 [==============================] - 3s 1ms/step - loss: 3.0125 - accuracy: 0.1788
Epoch 7/140
1857/1857 [==============================] - 3s 1ms/step - loss: 2.9031 - accuracy: 0.1955
Epoch 8/140
1857/1857 [==============================] - 3s 1ms/step - loss: 2.7883 - accuracy: 0.2132
Epoch 9/140
1857/1857 [==============================] - 3s 1ms/step - loss: 2.7256 - accuracy: 0.2246
Epoch 10/140
1857/1857 [==============================] - 3s 1ms/step - l

In [23]:
# Save model
model.save(os.path.join(path,'model_ens_3_all.h5'))

In [24]:
# Load model
# model = load_model(os.path.join(path,'model_ens_3_all.h5'))

In [25]:
# Preparing paths for test data
sub = pd.read_csv(os.path.join(path, "SampleSubmission.csv"))
sub['file_name'] = 'Test/'+sub['ID']+'.mp3'

In [26]:
X_test = features_extraction(sub['file_name'].values)

In [ ]:
#X_test = np.array(features_extraction(sub['file_name'].values[:50]))

In [0]:
# Write test features to csv
with open(os.path.join(path, "birds_test_276.csv"), "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(X_test)

In [28]:
# Read test features from csv
# X_test = np.array(pd.read_csv(os.path.join(path, "birds_test_276.csv"), header=None))

In [29]:
# Reshape test data
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

In [30]:
# Predict labels
Y_preds = model.predict([X_test[:,:128], X_test[:,128:134], X_test[:,134:276]])

In [36]:
# Transform probabilities to sample submission format
birds = sub.columns[1:41]
birdsn = lb.fit_transform(birds)

for ind, b in enumerate(birds):
  sub[b] = Y_preds[:, birdsn[ind]]

cols = ['ID']
for b in birds:
  cols.append(b)

ss = sub[cols]

In [37]:
# Write model's predicted probabilities to csv
ss.to_csv(os.path.join(path,'submission_starter_notebook_8.csv'), index=False)